# String Distance Measures

Implementation of various String Distance Measures in Python. This list is not extensive.

## Token-based
- Longest Common Substring Distance
- Jaccard Distance
- Overlap Coefficient
- Cosine

## Edit-based
- Hamming Distance
- Levenshtein Distance
- Needleman-Wunch Distance
- Optimal Damerau-Levenshtein Distance
- Jaro Distance
- Jaro-Winkler Distance

# Hybrid
- IBM (LCS-Levenshtein Normalized)
- Monge Elkan Distance

In [ ]:
# Longest Common Substring distance

def longest_common_string(string1, string2):
    if string1 == string2:
        return len(string1)
    
    rows = len(string1) + 1
    cols = len(string2) + 1
    table = [[0 for c in range(cols)] for r in range(rows)]

    longest = 0
    for col in range(cols):
        for row in range(rows):
            if col == 0 and row == 0:
                table[row][col] = 0
            if string1[row - 1] == string2[col - 1]:
                table[row][col] = table[row - 1][col - 1] + 1
                longest = max(longest, table[row][col])
            else:
                table[row][col] = 0
    
    return longest

assert longest_common_string('', '') == 0
assert longest_common_string('foobar', 'foobar') == 6
assert longest_common_string('foobar', 'foo') == 3
assert longest_common_string('foobar', 'f') == 1

In [ ]:
def ngrams(string, n=2):
    # N-Gram helper function
    return set([string[i:i + n] for i in range(0, len(string) - n + 1 )])

# Jaccard Distance

def jaccard(string1, string2):
    if string1 == string2:
        return 0.0
    
    ngrams1 = ngrams(string1)
    ngrams2 = ngrams(string2)
    
    unions = len(list(ngrams1.union(ngrams2)))
    qgrams = len(ngrams1.intersection(ngrams2))
    
    return 1 - qgrams / unions
    
assert jaccard('foobar', 'foobar') == 0.0
assert jaccard('foobar', 'foo') == 0.6
assert jaccard('foobar', 'xyz') == 1.0
assert jaccard('', '') == 0.0

# Overlap coefficient

def overlap(string1, string2):
    ngrams1 = ngrams(string1)
    ngrams2 = ngrams(string2)
    
    qgrams = len(ngrams1.intersection(ngrams2))

    return qgrams / len(min(ngrams1, ngrams2))

assert overlap('foobar', 'foobar') == 1.0
assert overlap('foo', 'foobar') == 1.0
assert overlap('barfoo', 'foobar') == 0.8
assert overlap('duane', 'dwayne') == 0.25

# Dice

def dice(string1, string2):
    ngrams1 = ngrams(string1)
    ngrams2 = ngrams(string2)
    
    qgrams = len(ngrams1.intersection(ngrams2))

    return 2 * ( qgrams / (len(ngrams1) + len(ngrams2)))

assert dice('foobar', 'foobar') == 1.0
assert dice('foo', 'foobar') == 0.5714285714285714
assert dice('barfoo', 'foobar') == 0.8
assert dice('duane', 'dwayne') == 0.2222222222222222

# Cosine

from math import sqrt

def cosine(string1, string2):
    
    if string1 == string2:
        return 1.0
    
    ngrams1 = ngrams(string1)
    ngrams2 = ngrams(string2)
    
    qgrams = len(ngrams1.intersection(ngrams2))

    return qgrams / (sqrt(len(ngrams1)) * sqrt(len(ngrams2)))

assert cosine('foobar', 'foobar') == 1.0
assert cosine('foobar', 'foo') == 0.6324555320336759
assert cosine('foobar', 'barfoo') == 0.7999999999999998

In [ ]:
# Hamming Distance
# Hint: Only for strings of equal length

def hamming(string1, string2):
    assert len(string1) == len(string2)
    return sum([char1 != char2 for char1, char2 in zip(string1, string2)])

assert hamming('', '') == 0
assert hamming('foobar', 'foobar') == 0
assert hamming('foobar', 'foubar') == 1
assert hamming('foobar', 'fuubar') == 2

In [ ]:
# Levenshtein distance

def levenshtein(string1, string2):

    if string1 == string2:
        return 0

    rows = len(string1) + 1
    cols = len(string2) + 1
    dist = [[0 for c in range(cols)] for r in range(rows)]

    for j in range(1, rows):
        dist[j][0] = j
    for i in range(1, cols):
        dist[0][i] = i

    for col in range(1, cols):
        for row in range(1, rows):
            cost = 1
            if string1[row - 1] == string2[col - 1]:
                cost = 0
            dist[row][col] = min(dist[row - 1][col] + 1, dist[row][col - 1] + 1, dist[row - 1][col - 1] + cost)

    # Enable for Debugging
    # print('\n'.join([''.join(['{:4}'.format(elem) for elem in row]) for row in dist]))
    return dist[row][col]

assert levenshtein('', '') == 0
assert levenshtein('foobar', 'foobar') == 0
assert levenshtein('foobar', 'foubar') == 1
assert levenshtein('foobar', 'fuubar') == 2
assert levenshtein('foobar', 'fuuar') == 3

In [ ]:
# Needleman Wunsch distance

def needleman_wunsch(string1, string2, gap_penalty=1):

    if string1 == string2:
        return len(string1)
    
    def match_score(char1, char2):
        return 1 if char1 == char2 else 0

    rows = len(string1) + 1
    cols = len(string2) + 1
    dist = [[0 for c in range(cols)] for r in range(rows)]

    for j in range(1, rows):
        dist[j][0] = -(j * gap_penalty)
    for i in range(1, cols):
        dist[0][i] = -(i * gap_penalty)
    
    for col in range(1, cols):
        for row in range(1, rows):
            match = dist[row - 1][col - 1] + match_score(string1[row - 1], string2[col - 1])
            delete = dist[row - 1][col] - gap_penalty
            insert = dist[row][col - 1] - gap_penalty
            dist[row][col] = max(match, delete, insert)
            
    return dist[row][col]

assert needleman_wunsch('', '') == 0
assert needleman_wunsch('foobar', 'foobar') == 6
assert needleman_wunsch('foobar', 'foo') == 0
assert needleman_wunsch('foobar', 'fo bar') == 5

In [ ]:
# Optimal Damerau-Levenshtein distance

def optimal_damerau_levenshtein(string1, string2):

    if string1 == string2:
        return 0
    
    rows = len(string1) + 1
    cols = len(string2) + 1
    dist = [[0 for c in range(cols)] for r in range(rows)]

    for j in range(1, rows):
        dist[j][0] = j
    for i in range(1, cols):
        dist[0][i] = i

    for col in range(1, cols):
        for row in range(1, rows):    
            cost = 1
            if string1[row - 1] == string2[col - 1]:
                cost = 0
            dist[row][col] = min(dist[row - 1][col] + 1, dist[row][col - 1] + 1, dist[row - 1][col - 1] + cost)
            
            if col > 1 and row > 1 and string1[row - 1] == string2[col - 2] and string1[row - 2] == string2[col - 1]:
                dist[row][col] = min(dist[row][col], dist[row - 2][col - 2] + cost)
            
    return dist[row][col]

assert optimal_damerau_levenshtein('', '') == 0
assert optimal_damerau_levenshtein('foobar', 'foobar') == 0
assert optimal_damerau_levenshtein('foobar', 'foubar') == 1
assert optimal_damerau_levenshtein('foobar', 'fuubar') == 2
assert optimal_damerau_levenshtein('foobar', 'fuuar') == 3
assert optimal_damerau_levenshtein('foobar', 'oofbraa') == 4

In [ ]:
# Jaro distance

def jaro(string1, string2):

    length1 = len(string1)
    length2 = len(string2)
   
    if length1 == 0:
        return 0.0
    
    if string1 == string2:
        return 1.0   

    match_bound = max(length1, length2) // 2 - 1

    matches = 0  
    transpositions = 0

    flagged_1 = [] 
    flagged_2 = []

    for i in range(length1):
        upperbound = min(i + match_bound, length2 - 1)
        lowerbound = max(0, i - match_bound)
        for j in range(lowerbound, upperbound + 1):
            if string1[i] == string2[j] and j not in flagged_2:
                matches += 1
                flagged_1.append(i)
                flagged_2.append(j)
                break

    flagged_2.sort()

    for i, j in zip(flagged_1, flagged_2):
        if string1[i] != string2[j]:
            transpositions += 1

    if matches == 0:
        return 0.0

    return (1/3 * ( matches / length1 + matches / length2 + (matches - transpositions // 2) / matches))

assert jaro('foobar', 'foobar') == 1.0
assert jaro('foobar', 'barfoo') == 0.4444444444444444
assert jaro('duane', 'dwayne') == 0.8222222222222222
assert jaro('hans', 'gruber') == 0.0

In [ ]:
# TODO Jaro-Winkler distance

def jaro_winkler(string1, string2):
     if string1 == string2:
        return 1.0

In [ ]:
# IBM (LCS-Levenshtein Normalized)
# Contractor, D., Faruquie, T. A., & Subramaniam, L. V. (2010, August). 

from itertools import groupby

def lcs_ratio(string1, string2):
    ratio = longest_common_string(string1, string2) / len(string1)
    return ratio

assert lcs_ratio('foobar', 'foobar') == 1.0
assert lcs_ratio('foo', 'bar') == 0.0
assert lcs_ratio('word', 'deoxyribonucleic') == 0.25


def consonant_skeleton(string, vowels='aeiouy'):
    without_vowels = ''.join([char for char in string if char not in vowels])     
    deduplicated_consonants = ''.join(char for char, _ in groupby(without_vowels))
    return deduplicated_consonants

assert consonant_skeleton('foobar') == 'fbr'
assert consonant_skeleton('ffoobbar') == 'fbr'
assert consonant_skeleton('barfoobar') == 'brfbr'


def ibm_similarity(string1, string2):
    similarity = lcs_ratio(string1, string2) / (levenshtein (consonant_skeleton(string1), consonant_skeleton(string2)) + 1)
    return similarity

assert ibm_similarity('foobar', 'foobar') == 1.0
assert ibm_similarity('foo', 'bar') == 0.0
assert ibm_similarity('word', 'deoxyribonucleic') == 0.03125

In [ ]:
# Monge-Elkan

def ngrams(string, n=2):
    # N-Gram helper function
    return set([string[i:i + n] for i in range(0, len(string) - n + 1 )])

def monge_elkan(string1, string2, similarity_function=levenshtein):
    if string1 == string2:
        return 1.0

    ngrams1 = ngrams(string1)
    ngrams2 = ngrams(string2)

    sum_of_maxes = 0
    for ngram1 in ngrams1:
        max_sim = float('-inf')
        for ngram2 in ngrams2:
             max_sim = max(max_sim, similarity_function(ngram1, ngram2))
        sum_of_maxes += max_sim
        
    return sum_of_maxes / len(ngrams1)

monge_elkan('foo', 'bar')
monge_elkan('foo', 'ksdfjsldk', similarity_function=optimal_damerau_levenshtein)